In [1]:
city_list = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", 
             "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes",
             "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

In [2]:
import requests
import pandas as pd
import json

In [3]:
# Ici je créée un dataframe contenant le nom de mes 35 sites
df = pd.DataFrame(city_list, columns=['City'])

In [4]:
# Ici je viens ajouter 3 colonnes à mon dataframe afin de stocker la donnée coloctée :
df['verif'] = ''
df['lat'] = ''
df['lon'] = ''

In [5]:
# Itération sur l'API nominatim :
url = 'https://nominatim.openstreetmap.org/search.php?'
#request = f'city=<{city}>&format=jsonv2'
lookups = ['display_name', 'lat', 'lon']

#loc_dict = []
city = df['City']

for i in range(len(df)):
    try:
        r = requests.get(url+f'q={city[i]}&countrycodes=FR&format=jsonv2').json()
        verif = r[0]['display_name']
        lat = round(float(r[0]['lat']), 2)
        #lat = requests.get(url+f'city=<{city}>&format=jsonv2')[1]['lat']
        lon = round(float(r[0]['lon']), 2)
        #lon = requests.get(url+f'city=<{city}>&format=jsonv2')[1]['lon']
        #print(f'{city[i]}, {verif}, {lat}, {lon}')
        df['lat'][i] = lat
        df['lon'][i] = lon
        df['verif'][i] = verif
    except:
        print('erreur')
        #df.drop(df.index[i], inplace=True)



In [14]:
# Ici j'ajoute mes colonnes moyenne précipitations pour les 7 prochains jours et moyenne de pluie pour les 7 prochains jours à mon dataframe
df['mean_7d_precipitation'] = ''
df['mean_7d_rain'] = ''
df['mean_7d_temp_min'] = ''
df['mean_7d_temp_max'] = ''

In [15]:
# Ici j'itère sur mon API afin de récupèrer les conditions méteos en termes de précipitation et de pluie pour chacun de mes sites, en utilisant la lattitude et la longitude
# des sites pour procéder au requettes 
# J'extraie ainsi la moyenne des valeurs cherchées pour les 7 prochains jours pour chaque site
lat = df['lat']
lon = df['lon']
for i in range(len(df)):
    try:
        r = requests.get(f'https://api.open-meteo.com/v1/forecast?latitude={lat[i]}&longitude={lon[i]}&daily=precipitation_sum,rain_sum,temperature_2m_min,temperature_2m_max&timezone=Europe%2FBerlin').json()
        df_meteo = pd.DataFrame(r['daily'])
        mean_7d_prep = round(df_meteo['precipitation_sum'].mean(),2)
        mean_7d_rain = round(df_meteo['rain_sum'].mean(), 2)
        mean_7d_temp_min = round(df_meteo['temperature_2m_min'].mean(), 2)
        mean_7d_temp_max = round(df_meteo['temperature_2m_max'].mean(), 2)

        df['mean_7d_precipitation'][i] = mean_7d_prep
        df['mean_7d_rain'][i] = mean_7d_rain
        df['mean_7d_temp_min'][i] = mean_7d_temp_min
        df['mean_7d_temp_max'][i] = mean_7d_temp_max
    except:
        print('erreur')
        #df.drop(df.index[i], inplace=True)

In [19]:
df.to_csv('src/meteo_villes_7j.csv')

In [16]:
lat = df['lat']
lon = df['lon']
city = df['City']

r = requests.get(f'https://api.open-meteo.com/v1/forecast?latitude={48.64}&longitude={-1.51}&daily=precipitation_sum,rain_sum,temperature_2m_min,temperature_2m_max&timezone=Europe%2FBerlin').json()
df_meteo_daily = pd.DataFrame(r['daily'])
df_meteo_daily['city'] = df['City'][0]
df_meteo_daily['lat'] = df['lat'][0]
df_meteo_daily['lon'] = df['lon'][0]

for i in range(1, len(df)):
    try:
        r = requests.get(f'https://api.open-meteo.com/v1/forecast?latitude={lat[i]}&longitude={lon[i]}&daily=precipitation_sum,rain_sum,temperature_2m_min,temperature_2m_max&timezone=Europe%2FBerlin').json()
        df_meteo_city = pd.DataFrame(r['daily'])
        df_meteo_city['city'] = city[i]
        df_meteo_city['lat'] = lat[i]
        df_meteo_city['lon'] = lon[i]

        df_meteo_daily = df_meteo_daily.append(df_meteo_city)

    except:
        print('erreur')

C:\Users\lucal\AppData\Local\Temp\ipykernel_19696\2218954982.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_meteo_daily = df_meteo_daily.append(df_meteo_city)
C:\Users\lucal\AppData\Local\Temp\ipykernel_19696\2218954982.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_meteo_daily = df_meteo_daily.append(df_meteo_city)
C:\Users\lucal\AppData\Local\Temp\ipykernel_19696\2218954982.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_meteo_daily = df_meteo_daily.append(df_meteo_city)
C:\Users\lucal\AppData\Local\Temp\ipykernel_19696\2218954982.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_meteo_daily = df_meteo_

In [25]:
df_meteo_daily = df_meteo_daily.reset_index()
df_meteo_daily.to_csv('src/meteo_villes_daily.csv')